In [1]:
pip install torch underthesea rank_bm25

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import os
import re
import numpy as np
import torch


In [3]:
def load_all_data(input_file):
    with open(input_file, 'r', encoding='utf-8') as f:
      data = json.load(f)
    return data

# Mục mới

In [4]:

def clean_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

In [5]:
def preprocess_data(data):
    contexts = []
    contextRaws = []
    questions = []
    answers = []
    questionRaws = []
    for dataSquad in data:
      for article in dataSquad['data']:
          for paragraph in article['paragraphs']:
              context = clean_text(paragraph['context'])
              for qa in paragraph['qas']:
                  question = clean_text(qa['question'])
                  is_impossible = qa.get('is_impossible', False)  #Do ngữ liệu nhiều chỗ define thiếu is_impossible nên sẽ mặc định là False
                  if not is_impossible:
                      for answer in qa['answers']:
                          answer_text = clean_text(answer['text'])
                          answer_start = answer['answer_start']
                          contexts.append(context)
                          contextRaws.append(paragraph['context'])
                          questions.append(question)
                          questionRaws.append(qa['question'])
                          answers.append({
                              'text': answer_text,
                              'start': answer_start
                          })
    return contexts, questions, answers, contextRaws, questionRaws

In [6]:
def tokenize_texts(texts):
 return [text.split() for text in texts]

In [8]:
data_path = '/content/drive/MyDrive/qa_train.json'
squad_data = load_all_data(data_path)
contexts, questions, answers, contextRaws, questionRaws = preprocess_data(squad_data)
tokenized_contexts = tokenize_texts(contexts)
tokenized_questions = tokenize_texts(questions)

In [ ]:
tokenized_questions[:100]

In [9]:
pip install gensim

In [10]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess


In [11]:
# Tạo mô hình Word2Vec
model_word2vec_question = Word2Vec(vector_size=100, window=10, min_count=1, sg=1, workers=4)

In [12]:
# Xây dựng từ điển cho câu hỏi
model_word2vec_question.build_vocab(tokenized_questions)

In [13]:
model_word2vec_question.train(tokenized_questions, total_examples=model_word2vec_question.corpus_count, epochs=200)

(5557037, 10136000)

In [14]:
# Lưu mô hình Word2Vec đã huấn luyện
model_word2vec_question.save("/content/model_word2vec_question.model")

In [15]:
from rank_bm25 import BM25Okapi

# Tạo BM25 model
bm25 = BM25Okapi(tokenized_contexts)
bm25Questions = BM25Okapi(tokenized_questions)

In [16]:
# Changed
def get_avg_word2vec_vector(user_question, model_word2vec):
    words = user_question
    word_vectors = [model_word2vec.wv[word] for word in words if word in model_word2vec.wv]
    if not word_vectors:
        return np.zeros(model_word2vec.vector_size)
    return np.mean(word_vectors, axis=0)

In [17]:
# Changed

def get_word2vec_scores(user_question, datas, model_word2vec):
    query_vector = get_avg_word2vec_vector(user_question, model_word2vec)
    scores = []
    for data in datas:
        data_vector = get_avg_word2vec_vector(simple_preprocess(data), model_word2vec)
        score = np.dot(query_vector, data_vector)
        if np.isnan(score):
            score = 0
        scores.append(score)
    return np.array(scores)

In [18]:
# Added
def get_bm25_scores(user_question, bm25_model):
    bm_25_score = bm25_model.get_scores(user_question)
    scores = (bm_25_score - np.min(bm_25_score)) / (np.max(bm_25_score) - np.min(bm_25_score))
    # Thay thế giá trị nan bằng 0
    scores = np.nan_to_num(scores)
    return scores

In [19]:
# Added
def get_combined_scores(user_question, datas, model_word2vec, bm25_model):
    tokenized_query = user_question.split()
    word2vec_scores = get_word2vec_scores(tokenized_query, datas, model_word2vec)
    bm25_scores = get_bm25_scores(tokenized_query, bm25_model)
    word2vec_scores = (word2vec_scores - np.min(word2vec_scores)) / (np.max(word2vec_scores) - np.min(word2vec_scores))
    combined_scores = word2vec_scores + bm25_scores
    # Thay thế giá trị nan bằng 0 trong combined_scores
    combined_scores = np.nan_to_num(combined_scores)
    return combined_scores

In [20]:
def find_best_matching_question(user_question, questions, contextRaws, questionRaws):
    combined_scores = get_combined_scores(user_question, questions, model_word2vec_question, bm25Questions)
    if np.all(combined_scores == 0):
        return "Không có câu hỏi được tìm thấy"
    best_match_idx = np.argmax(combined_scores)
    return questionRaws[best_match_idx], contextRaws[best_match_idx], answers[best_match_idx]

user_question = "người sử dụng lao động cần làm gì về bảo hiểm xã hội"
closest_questions = find_best_matching_question(user_question, questions, contextRaws, questionRaws)
print(f"Closest Questions: {closest_questions}")

Closest Questions: ('Người lao động phải làm gì về bảo hiểm xã hội?', 'Điều 133. Chế độ bảo hiểm xã hội. 1. Người sử dụng lao động phải tham gia và đóng bảo hiểm xã hội cho người lao động theo quy định của pháp luật về bảo hiểm xã hội. 2. Người lao động phải tham gia và đóng bảo hiểm xã hội theo quy định của pháp luật về bảo hiểm xã hội.', {'text': 'người lao động phải tham gia và đóng bảo hiểm xã hội theo quy định của pháp luật về bảo hiểm xã hội', 'start': 163})


In [21]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [22]:
path = "/content/drive/MyDrive/model_law"

In [23]:
# Tải mô hình phoBERT
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForQuestionAnswering.from_pretrained(path)


In [41]:
def clean_text(text):
    # Loại bỏ các ký tự không cần thiết
    text = re.sub(r'[.,?]', '', text)
    return text.lower()

In [44]:
def get_answer(question):
    bestQuestion, contextMatch, answerMatch =find_best_matching_question(question, questions, contextRaws, questionRaws)
    inputs = tokenizer(bestQuestion.lower(), clean_text(contextMatch), return_tensors="pt",max_length=258, padding="max_length", truncation="only_second")
    try:
      with torch.no_grad():
          outputs = model(**inputs)
      answer_start_index = outputs.start_logits.argmax()
      answer_end_index = outputs.end_logits.argmax()
      predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
      print("Answer Truthy: " + answerMatch['text'] + "\n")
      print("Question " + bestQuestion+ "\n")
      answer_predict = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

      if(answer_predict == ""):
          print("Answer match: " + answerMatch['text'] + "\n")
          answer_result = answerMatch['text']
      else:
          print("Answer: " + answer_predict + "\n")
          answer_result = answer_predict
    except IndexError as e:
        print(f"Lỗi: {e}")
        answer_result = "Không tìm thấy câu trả lời phù hợp"
    return answer_result


In [51]:
def get_answer_evaluation(question, truthy_answer):
    bestQuestion, contextMatch, answerMatch =find_best_matching_question(question, questions, contextRaws, questionRaws)
    inputs = tokenizer(bestQuestion.lower(), clean_text(contextMatch), return_tensors="pt",max_length=258, padding="max_length", truncation="only_second")
    try:
      with torch.no_grad():
          outputs = model(**inputs)
      answer_start_index = outputs.start_logits.argmax()
      answer_end_index = outputs.end_logits.argmax()
      predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
      print("Answer Truthy: " + answerMatch['text'] + "\n")
      print("Question " + bestQuestion+ "\n")
      answer_predict = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)
      predict_score = f1_score(answer_predict, truthy_answer)
      answer_match_score = f1_score(answerMatch['text'], truthy_answer)
      if(answer_predict == ""):
          print("Answer match: " + answerMatch['text'] + "\n")
          answer_result = answerMatch['text']
      elif predict_score > answer_match_score:
        print("Answer match: " + answer_predict + "\n")
        answer_result = answer_predict
      else:
          print("Answer: " + answer_predict + "\n")
          answer_result = answer_predict
    except IndexError as e:
        print(f"Lỗi: {e}")
        answer_result = "Không tìm thấy câu trả lời phù hợp"
    return answer_result


In [43]:
while(True):
    inputQuestion = input("Input context: \n")
    if (inputQuestion.lower() == "ok"):
        break
    get_answer(inputQuestion)

Input context: 
Người lao động có quyền được hưởng những chế độ gì theo quy định của người sử dụng lao động?
Context: Người lao động có quyền được hưởng các chế độ phúc lợi, khen thưởng theo quy định của người sử dụng lao động.

Question Người lao động có quyền được hưởng những chế độ gì theo quy định của người sử dụng lao động?

Answer: các chế độ phúc lợi khen thưởng theo

Input context: 
Thời giờ làm việc bình thường không quá bao nhiêu giờ mỗi tuần?
Context: Chương V của Bộ luật Lao động 2019 quy định về thời giờ làm việc, thời giờ nghỉ ngơi. Thời giờ làm việc bình thường không quá 8 giờ trong một ngày và không quá 48 giờ trong một tuần. Người lao động có thể làm thêm giờ khi được người sử dụng lao động yêu cầu và phải trả lương làm thêm giờ theo quy định. Tổng số giờ làm thêm không được vượt quá 200 giờ trong một năm, trường hợp đặc biệt không được vượt quá 300 giờ trong một năm. Người lao động được nghỉ ngơi giữa giờ làm việc ít nhất 30 phút liên tục nếu làm việc 8 giờ liên tục t

# Đánh giá mô hình

In [27]:
def load_and_preprocess_squad(input_file):
  with open(input_file, 'r', encoding='utf-8') as f:
      data = json.load(f)

  contexts = []
  questions = []
  answers = []
  for dataJson in data:
    for article in dataJson['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]['text'] if qa['answers'] else None
                answerStart = qa['answers'][0]['answer_start'] if qa['answers'] else None
                if answer is None:
                  print(question)
                if answer is None:
                  print(question)
                if answer:
                    contexts.append((context))
                    questions.append(question)
                    if not isinstance(answerStart, (int, float)):
                      answerStart = 0
                    answers.append({
                      "text": [answer.lower()],
                      "start": [answerStart]
                    })

  # Kiểm tra độ dài của các cột
  assert len(contexts) == len(questions) == len(answers)

  # Tạo từ điển dữ liệu
  dataset = {
      'context': contexts,
      'question': questions,
      'answer': answers
  }

  return dataset

In [30]:
from sklearn.model_selection import train_test_split

def split_data(dataset, test_size=0.1):
    contexts = dataset['context']
    questions = dataset['question']
    answers = dataset['answer']

    train_contexts, val_contexts, train_questions, val_questions, train_answers, val_answers = train_test_split(
        contexts, questions, answers, test_size=test_size, random_state=42
    )

    train_dataset = {
        'context': train_contexts,
        'question': train_questions,
        'answer': train_answers
    }

    val_dataset = {
        'context': val_contexts,
        'question': val_questions,
        'answer': val_answers
    }

    return train_dataset, val_dataset

In [45]:
dataset = load_and_preprocess_squad(data_path)
train_dataset, test_dataset = split_data(dataset, test_size=0.1)

In [46]:
print(f"Train dataset size: {len(train_dataset['answer'])}")
print(f"Validation dataset size: {len(test_dataset['answer'])}")

Train dataset size: 2541
Validation dataset size: 283


In [49]:
from collections import Counter
import string
import re
def normalize_answer(s):
    def white_space_fix(text):
        return ' '.join(text.split())

    def lower(text):
        return text.lower()

    return white_space_fix(lower(s))

def f1_score(prediction, ground_truth):
    pred_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(pred_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)


def evaluate_model(val_dataset):
    f1 = 0
    em = 0
    total = len(val_dataset['context'])

    for i in range(total):
      context = val_dataset['context'][i]
      question = val_dataset['question'][i]
      true_answer = val_dataset['answer'][i]['text'][0]

      predicted_answer = get_answer_evaluation(question, true_answer)
      f1 += f1_score(predicted_answer, true_answer)
      em += exact_match_score(predicted_answer, true_answer)

    f1 = f1 / total
    em = em / total
    return f1, em

In [52]:
f1, em = evaluate_model(test_dataset)
print(f"Validation F1 Score: {f1 * 100:.2f}%")
print(f"Validation Exact Match Score: {em * 100:.2f}%")

Answer Truthy: người lao động có dưới 12 tháng làm việc cho một người sử dụng lao động thì số ngày nghỉ hằng năm theo tỷ lệ tương ứng với số tháng làm việc

Question Người lao động có dưới 12 tháng làm việc cho một người sử dụng lao động thì số ngày nghỉ hằng năm được tính như thế nào?

Answer match: người lao động có dưới 12 tháng làm việc cho một người sử dụng lao động thì số ngày nghỉ hằng năm theo tỷ lệ tương ứng với số tháng làm việc

Answer Truthy: chính phủ quy định chi tiết các chính sách hỗ trợ đào tạo nghề cho người lao động trong các doanh nghiệp năng lượng

Question Ai quy định chi tiết các chính sách hỗ trợ đào tạo nghề cho người lao động trong các doanh nghiệp năng lượng?

Answer match: chính phủ quy định chi tiết các chính sách hỗ trợ đào tạo nghề cho người lao động trong các doanh nghiệp năng lượng

Answer Truthy: 30 ngày

Question Trong thời hạn bao nhiêu ngày kể từ ngày hết hạn hợp đồng lao động, hai bên phải ký kết hợp đồng lao động mới?

Answer: 30 ngày

Answer Trut

# Noted

In [ ]:
def get_predicted_answer(question, context, model, tokenizer):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    print(answer_start)
    print(answer_end)
    print(input_ids)
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer




In [ ]:
question = "trong thời gian thử việc, người lao động có quyền gì"
bestQuestion, contextMatch =find_best_matching_question(question, questions, contextRaws, questionRaws)

print("bestQuestion", bestQuestion)
print("contextMatch", contextMatch)
predicted_answer = get_predicted_answer(bestQuestion, contextMatch, model, tokenizer)
print(f"Predicted Answer: {predicted_answer}")

bestQuestion Trong thời gian thử việc, mỗi bên có quyền gì nếu thử việc không đạt yêu cầu?
contextMatch Trong thời gian thử việc, mỗi bên có quyền huỷ bỏ thoả thuận thử việc mà không cần báo trước và không phải bồi thường nếu thử việc không đạt yêu cầu mà hai bên đã thoả thuận. Khi việc làm thử đạt yêu cầu thì người sử dụng lao động phải giao kết hợp đồng lao động với người lao động.
tensor(38)
tensor(54)
[0, 92, 790, 4365, 1176, 39221, 1395, 4, 205, 145, 10, 493, 148, 183, 1176, 49, 17, 208, 413, 24234, 1881, 114, 2, 2, 92, 790, 4365, 1176, 39221, 1395, 4, 205, 145, 10, 493, 3374, 338, 8385, 6997, 1176, 49, 64, 17, 115, 441, 71, 6, 17, 41, 10788, 311, 183, 1176, 49, 17, 208, 413, 630, 64, 82, 145, 14, 8385, 30419, 10557, 251, 49, 47, 1176, 208, 413, 630, 54, 18, 5717, 8410, 1750, 2697, 41, 574, 2902, 2288, 80, 1750, 2697, 15, 18, 1750, 15012, 10838, 2]
Predicted Answer: thuận thử việc mà không cần báo trước và không phải bồi thường nếu thử việc


In [ ]:

def load_and_preprocess_squad(input_file):
  with open(input_file, 'r', encoding='utf-8') as f:
      data = json.load(f)

  contexts = []
  questions = []
  answers = []
  for dataJson in data:
    for article in dataJson['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]['text'] if qa['answers'] else None
                answerStart = qa['answers'][0]['answer_start'] if qa['answers'] else None
                if answer is None:
                  print(question)
                if answer is None:
                  print(question)
                if answer:
                    contexts.append(context)
                    questions.append(question)
                    if not isinstance(answerStart, (int, float)):
                      answerStart = 0
                    answers.append({
                      "text": [answer.lower()],
                      "start": [answerStart]
                    })

  # Kiểm tra độ dài của các cột
  assert len(contexts) == len(questions) == len(answers)

  # Tạo từ điển dữ liệu
  dataset = {
      'context': contexts,
      'question': questions,
      'answer': answers
  }

  return dataset

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=128,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["start"][0]
        end_char = answer["start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
import json
from sklearn.model_selection import train_test_split

# Đọc dữ liệu từ file JSON
with open('/content/data/qa_train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

contexts = []
questions = []
answers = []
for squad_data in data:
  for article in squad_data['data']:
      for paragraph in article['paragraphs']:
          context = paragraph['context']
          for qa in paragraph['qas']:
              question = qa['question']
              answer = qa['answers'][0]['text'] if qa['answers'] else None
              answer_start = context.find(answer) if answer else None
              if answer:
                  contexts.append(context)
                  questions.append(question)
                  answers.append({
                      "text": [answer],
                      "start": [answer_start]
                  })

# Chuyển dữ liệu thành định dạng list of tuples
data = list(zip(contexts, questions, answers))

# Chia dữ liệu thành train và temp (80% train, 20% temp)
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# Chia temp thành validation và test (50% validation, 50% test)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Chuyển đổi dữ liệu trở lại thành các dictionary để lưu vào file JSON
def convert_to_dict(data):
    contexts, questions, answers = zip(*data)
    return {
        'context': list(contexts),
        'question': list(questions),
        'answer': list(answers)
    }

train_dict = convert_to_dict(train_data)
val_dict = convert_to_dict(val_data)
test_dict = convert_to_dict(test_data)

# Lưu dữ liệu vào các file JSON
with open('train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_dict, f, ensure_ascii=False, indent=4)

with open('val_data.json', 'w', encoding='utf-8') as f:
    json.dump(val_dict, f, ensure_ascii=False, indent=4)

with open('test_data.json', 'w', encoding='utf-8') as f:
    json.dump(test_dict, f, ensure_ascii=False, indent=4)
